In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time


In [9]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Initialize OpenCV video capture
cap = cv2.VideoCapture(0)

# Constants and counters for EAR and MAR
EAR_CONSEC_FRAMES = 30
EAR_ALERT_THRESHOLD = 5
ALERT_DISPLAY_FRAMES = 60

# Variables for tracking the state
ear_frame_count = 0
ear_alert_count = 0
alert_frame_counter = 0
display_alert = False

# FPS calculation
prev_time = time.time()

# EAR calculation function
def calculate_ear(landmarks, eye_indices):
    left_point = np.array([landmarks[eye_indices[0]].x, landmarks[eye_indices[0]].y])
    right_point = np.array([landmarks[eye_indices[3]].x, landmarks[eye_indices[3]].y])
    top_mid = np.array([(landmarks[eye_indices[1]].x + landmarks[eye_indices[2]].x) / 2,
                        (landmarks[eye_indices[1]].y + landmarks[eye_indices[2]].y) / 2])
    bottom_mid = np.array([(landmarks[eye_indices[4]].x + landmarks[eye_indices[5]].x) / 2,
                           (landmarks[eye_indices[4]].y + landmarks[eye_indices[5]].y) / 2])

    horizontal_distance = np.linalg.norm(right_point - left_point)
    vertical_distance = np.linalg.norm(top_mid - bottom_mid)

    ear = vertical_distance / horizontal_distance
    return ear


# Calculate brightness
def calculate_brightness(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return np.mean(gray_frame)

# Adjust EAR threshold based on brightness
def adjust_ear_threshold(brightness):
    if brightness < 50:
        return 0.18
    elif brightness < 100:
        return 0.21
    else:
        return 0.24
    
# Indices for left and right eyes  (MediaPipe Face Mesh)
LEFT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_INDICES = [362, 385, 387, 263, 373, 380]


try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame. Exiting...")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)

        h, w, _ = frame.shape

        # Calculate brightness and EAR threshold
        brightness = calculate_brightness(frame)
        ear_threshold = adjust_ear_threshold(brightness)

        # FPS calculation
        current_time = time.time()
        fps = 1 / (current_time - prev_time)
        prev_time = current_time

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                landmarks = face_landmarks.landmark
                
                # Draw circles on left and right eye landmarks
                for idx in LEFT_EYE_INDICES + RIGHT_EYE_INDICES:
                    x = int(landmarks[idx].x * w)
                    y = int(landmarks[idx].y * h)
                    cv2.circle(frame, (x, y), 2, (0, 200, 200), -1)

                # Calculate EAR and MAR
                left_ear = calculate_ear(landmarks, LEFT_EYE_INDICES)
                right_ear = calculate_ear(landmarks, RIGHT_EYE_INDICES)
                avg_ear = (left_ear + right_ear) / 2

                # EAR detection logic
                if avg_ear < ear_threshold:
                    ear_frame_count += 1
                    if ear_frame_count == EAR_CONSEC_FRAMES:
                        ear_alert_count += 1
                        ear_frame_count = 0
                else:
                    ear_frame_count = 0
                    
        # Trigger final alert if thresholds are reached
        if ear_alert_count >= EAR_ALERT_THRESHOLD:
            alert_frame_counter = ALERT_DISPLAY_FRAMES
            ear_alert_count = 0
            display_alert = True
            
        # Decrement alert frame counter
        if alert_frame_counter > 0:
            alert_frame_counter -= 1
        else:
            display_alert = False
            
        # Draw semi-transparent box with a small black border for overlay information
        overlay = frame.copy()
        cv2.rectangle(overlay, (5, 5), (180, 100), (96, 96, 96), -1)  # Semi-transparent box
        alpha = 0.6
        frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)  # Blend overlay with frame
        cv2.rectangle(frame, (5, 5), (180, 100), (0, 0, 0), 2)  # Black border
        
         # Display information on frame
        cv2.putText(frame, f"EAR COUNT: {ear_frame_count}", (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.putText(frame, f"EAR ALERTS: {ear_alert_count}", (15, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
        cv2.putText(frame, f"FPS: {fps:.2f}", (15, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        if display_alert:
            cv2.putText(frame, "ALERT: DROWSINESS DETECTED!", (w // 2 - 250, h // 2), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

        # Resize the frame to fit the window
        frame = cv2.resize(frame, (780, 540))  # Set to the same size as the window
        cv2.imshow("Drowsiness Detection", frame)
        cv2.namedWindow("Drowsiness Detection", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Drowsiness Detection", 780, 540)  # Set the window size (width x height)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cap.release()
    cv2.destroyAllWindows()